<a href="https://colab.research.google.com/github/aadhyathakur2211/summary_generator/blob/main/imagepreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=da45cd7bce992ca6d42e39b62b89ec452c54c53606d28e7221c1c0b18c16f9b4
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c303e8ff175416d5c1c9c03a2e83ebf7525a99
Successfully built openai


In [2]:
!pip install --ignore-installed Pillow==9.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 59.3 MB/s eta 0:00:00


In [5]:
from google.colab import files
# Upload the image file to Colab
uploaded = files.upload()


Saving testimage.png to testimage.png


In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# load the image
img = cv2.imread("testimage.png")

#convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# APPLYING GAUSSIAN BLUR TO REMOVE NOISE
blur = cv2.GaussianBlur(gray, (5,5), 0)

# applying image threshold to extract the graph
ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Finding the contours in the image
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#Finding the largest contour
largest_contour = max(contours, key=cv2.contourArea)

#finding the bounded rectangle 
x,y,w,h = cv2.boundingRect(largest_contour)

#extracting the graph region
graph = img[y:y+h, x:x+w]

#applying edge detection to extract the edges of the graph
edges = cv2.Canny(graph, 100, 200)

#Applying Hough Line Transform to extract the lines in the graph
lines = cv2.HoughLines(edges, 1, np.pi/180, 200)

#extracting x and y axis from the lines
x_axis = None
y_axis = None
for line in lines:
    rho, theta = line[0]
    if theta < np.pi/4 or theta > 3*np.pi/4:
        if x_axis is None or rho > x_axis[0]:
            x_axis = (rho, theta)
    else:
        if y_axis is None or rho > y_axis[0]:
            y_axis = (rho, theta)

#extracting the data points from the graph
data_points = []
for contour in contours:
    area = cv2.contourArea(contour)
    if area > 10:
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            if y_axis is not None and x_axis is not None:
                if y_axis[1] < np.pi/4 or y_axis[1] > 3*np.pi/4:
                    value = (cx - x_axis[0]) / np.cos(x_axis[1])
                    data_points.append((cx, cy, value))
                else:
                    value = (y_axis[0] - cy) / np.sin(y_axis[1])
                    data_points.append((cx, cy, value))
      






TypeError: ignored